In [19]:
from visualization_fct import *
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import scale

# from bokeh.plotting import output_file, show, save
# from bokeh.mpl import to_bokeh

from bokeh.resources import CDN
from bokeh.embed import file_html
from bokeh.plotting import output_notebook, show, output_file
output_notebook()

import matplotlib.pyplot as plt  # , mpld3
import pyfits


np.random.seed(0)

# for now, no ratio in data (no rates A, B or C in this dataset)

only_flux = True
scale_flux = False

hdulist = pyfits.open('../iirc_data/all_data_for_ml.fits')
data = hdulist[1].data

# we need to change the byte order for fits -> dataframe:
if scale_flux:
    data_fr = pd.DataFrame(scale(data['flux'].byteswap().newbyteorder()))
else:
    data_fr = pd.DataFrame(data['flux'].byteswap().newbyteorder())
# convert attributes to str:
data_fr.columns = [str(j) for j in data_fr.columns]


if not only_flux:

    names = [data.columns[j].name for j in range(len(data.columns))]

    # rm str attribute:
    names.remove('block')
    names.remove('orbitalphase')
    names.remove('smoothorbitalphase')

    # for now, rm multidim attributes:
    names.remove('en_lo')
    names.remove('en_hi')
    names.remove('flux')  # already in
    names.remove('flux_err')  # error term

    # also remove error terms, tstart and tstop:
    #names.remove('rms1')
    #names.remove('rms2')
    #names.remove('rms3')
    #names.remove('rms4')
    names.remove('tstart')
    names.remove('tstop')

if only_flux:
    names = ['gamma']  # add only the labels (labels derive from gamma)
    
# we need to change the byte order for fits -> dataframe:
data_fr2 = pd.DataFrame({name: data[name].byteswap().newbyteorder()
                         for name in names})

data_fr = pd.concat([data_fr, data_fr2], axis=1)

# rm the rows with nan values:
data_fr = data_fr.dropna()

# rm too large values except for 'orbit':
if only_flux:
    data_thr = mask(data_fr, ['gamma'])
else:
    data_thr = mask(data_fr, ['orbitalphase', 'smoothorbitalphase','gamma'])


# labels from gamma as in the paper:
a = np.array(data_thr['gamma'])
y = (a > 2.5).astype('int') + (a > 2).astype('int')
col_without_gamma = list(data_thr.columns)
col_without_gamma.remove('gamma')

X = np.concatenate([np.array(data_thr[name]).reshape(-1, 1) for name in col_without_gamma], axis=1)

Loading BokehJS ...

In [20]:
# make video / widget of showing spectra over time

en = (data['en_lo'].byteswap().newbyteorder() + data['en_hi'].byteswap().newbyteorder()) / 2.

data_fr_en = pd.DataFrame(en)

# convert attributes to str:
data_fr_en.columns = [str(j) for j in data_fr_en.columns]

#rm nan:
data_fr_en = data_fr_en.dropna()

#check same axis shape than data_fr (ie nan correspond to same lines):
print data_fr_en.shape[0] == data_fr.shape[0]

En = np.concatenate([np.array(data_fr_en[name]).reshape(-1, 1) for name in data_fr_en.columns], axis=1)

True


In [25]:
X.shape

(2713, 78)

In [24]:
import matplotlib.animation as animation

#%matplotlib auto  #  'osx', 'qt4', 'qt5', 'gtk3', 'wx', 'qt', 'gtk', 'tk'

fig, ax = plt.subplots()

#line, = ax.plot(En[0, :], X[0, :])
line, = ax.plot(range(X.shape[1]), X[0, :])

def animate(i):
    # line.set_xdata(En[i, :])  # update the data
    line.set_ydata(X[i, :])   # update the data
    return line,


## Init only required for blitting to give a clean slate.
#def init():
#    line.set_ydata(np.ma.array(x, mask=True))
#    return line,

ani = animation.FuncAnimation(fig, animate, np.arange(1, 2000), #init_func=init,
                              interval=250, blit=True)
plt.show()


#ani.save('spectra_over_time.mp4', writer = 'mencoder', fps=30, extra_args=['-vcodec', 'libx264'])

In [6]:
y

array([0, 0, 0, ..., 2, 2, 2])

In [7]:
# visualize datapoints by plotting spectrum multiplied by energy (en_hi + en_lo)/2
colors = ['red', 'blue', 'yellow']

fig, ax = plt.subplots()
#plt.axes(xlim=(0, 1e-6), ylim=(0, 80))

line, = ax.plot(En[0, :], X[0, :] * En[0, :])
#line, = ax.plot(range(X.shape[1]), X[0, :] * En[0, :])

def animate(i):
    line.set_xdata(En[i, :])  # update the data
    line.set_ydata(X[i, :] * En[i, :])   # update the data
    line.set_color(colors[y[i]])
    return line,


# Init only required for blitting to give a clean slate.
def init():
    line.set_xdata(En[0, :])
    line.set_ydata(X[0, :] * En[0, :])
    line.set_color(colors[y[0]])
    return line,

ani = animation.FuncAnimation(fig, animate, np.arange(1, 2000), init_func=init,
                              interval=25, blit=True)
plt.show()

ValueError: operands could not be broadcast together with shapes (89,) (78,) 

In [ ]:
lda = LDA(n_components=2).fit(X, y)
preds = lda.predict(X)
X_lda = lda.transform(X)
X_lda

In [ ]:
%matplotlib inline
clr = np.array(['blue', 'red', 'yellow'])
for j in range(3):
    plt.plot(X_lda[:, 0][y==j], X_lda[:, 1][y==j], '.')
plt.show()

In [ ]:
for j in range(3):
    plt.plot(X_lda[:, 0][preds==j], X_lda[:, 1][preds==j], '.')
plt.show()

In [ ]:
lda.explained_variance_ratio_

In [ ]:
plt.plot(lda.means_.T)
plt.show()

In [ ]:
plt.plot(lda.scalings_)
plt.show()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_pca = pca.fit(X).transform(X)
%matplotlib notebook
plt.figure()
for j in range(3):
    plt.plot(X_pca[:, 0][y==j], X_pca[:, 1][y==j], '.')
plt.show()

In [ ]:
pca.explained_variance_ratio_

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=None)
X_pca = pca.fit(X).transform(X)
plt.figure()
print('pca.explained_variance_ratio_', pca.explained_variance_ratio_)
#df = pd.DataFrame(X_pca, columns=['1','2','3'])
#df['y'] = pd.Series(y).astype("category")
#scatter_matrix_seaborn(df, y='y', size=3)
plt.semilogy(pca.explained_variance_ratio_)
plt.show()

In [ ]:
#pca.components_
plt.figure()
plt.plot(pca.components_[:4].T)
plt.show()

In [ ]:
#nmf:

from sklearn.decomposition import NMF
Xpos = X - X.min(axis=0)
model = NMF(n_components=2, init='random', random_state=0)
model.fit(Xpos) 
X_nmf = model.transform(Xpos)
for j in range(3):
    plt.plot(X_nmf[:, 0][y==j], X_pca[:, 1][y==j], '.')
plt.show()
print 'model.reconstruction_err_', model.reconstruction_err_
#print 'model.components_', model.components_

In [ ]:
model = NMF(n_components=2, init='random', random_state=0)
%matplotlib notebook
plt.figure()
model.fit(Xpos)
X_nmf = model.transform(Xpos)
for j in range(3):
    plt.plot(X_nmf[:, 0][preds==j], X_nmf[:, 1][preds==j], '.', c='k')
plt.show()

In [ ]:

plt.figure()
plt.plot(model.components_.T)
plt.show()

In [ ]:
model = NMF(n_components=3, init='random', random_state=0)
model.fit(Xpos)

X_nmf = model.transform(Xpos)
df = pd.DataFrame(X_nmf, columns=['1','2', '3'])
df['y'] = pd.Series(y).astype("category")
scatter_matrix_seaborn(df, y='y', size=3)
print 'model.reconstruction_err_', model.reconstruction_err_
#print 'model.components_', model.components_

In [ ]:
model = NMF(n_components=4, init='random', random_state=0)
model.fit(Xpos) 
X_nmf = model.transform(Xpos)
df = pd.DataFrame(X_nmf, columns=['1','2', '3', '4'])
df['y'] = pd.Series(y).astype("category")
scatter_matrix_seaborn(df, y='y', size=3)
print 'model.reconstruction_err_', model.reconstruction_err_
#print 'model.components_', model.components_

In [ ]:
err = []
n_comps = range(1, 11)
for n_comp in n_comps:
    model = NMF(n_components=n_comp, init='random', random_state=0)
    model.fit(Xpos) 
    err += [model.reconstruction_err_]
plt.plot(n_comps, err)